In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv("Crop_recommendation.csv")

# Train Test Split

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(columns='label'),
    df['label'],
    test_size=0.2
)

# column Transformers

In [11]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [13]:
trf1 = ColumnTransformer(transformers=[
    ('standarisation', StandardScaler(with_mean=False), [0,6]),
],
    remainder='passthrough'
)

In [15]:
trf2 = ColumnTransformer(transformers=[
    ('ohe_label', OneHotEncoder(sparse=False, handle_unknown='ignore'), [-1])
], remainder='passthrough')

# importing models

In [20]:
from sklearn.neighbors import KNeighborsClassifier  # k-Nearest Neighbors (KNN)
from sklearn.svm import SVC  # Support Vector Machine (SVM)
from sklearn.tree import DecisionTreeClassifier  # Decision Tree Classifier
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.naive_bayes import GaussianNB  # Naive Bayes
from sklearn.ensemble import GradientBoostingClassifier  # Gradient Boosting Machine (GBM)
from sklearn.ensemble import RandomForestClassifier  # Random Forest Classifier


In [24]:
models_dict = {
    'KNeighborsClassifier': KNeighborsClassifier(),
    'SVC': SVC(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'LogisticRegression': LogisticRegression(),
    'GaussianNB': GaussianNB(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
}


In [26]:
models_dict.keys()

dict_keys(['KNeighborsClassifier', 'SVC', 'DecisionTreeClassifier', 'LogisticRegression', 'GaussianNB', 'GradientBoostingClassifier', 'RandomForestClassifier'])

In [34]:
pipelines = {
    'KNeighborsClassifier': Pipeline([
        ('scaler', trf1) , # StandardScaler as a preprocessing step
        ('model', KNeighborsClassifier())
    ]),
    'SVC': Pipeline([
        ('scaler', trf1),
        ('model', SVC())
    ]),
    'DecisionTreeClassifier': Pipeline([
        # No need to scale for Decision Tree, so skipping the scaler
        ('encode', trf2),
        ('model', DecisionTreeClassifier())
    ]),
    'LogisticRegression': Pipeline([
        ('scaler', StandardScaler()),
        ('encode', trf2),
        ('model', LogisticRegression())
    ]),
    'GaussianNB': Pipeline([
        # Naive Bayes does not require scaling, skipping the scaler
        ('encode', trf2),
        ('model', GaussianNB())
    ]),
    'GradientBoostingClassifier': Pipeline([
        # Skipping scaler for GBM as it doesn't need feature scaling
        ('encode', trf2),
        ('model', GradientBoostingClassifier())
    ]),
    'RandomForestClassifier': Pipeline([
        # Skipping scaler for Random Forest as it's not necessary
        ('encode', trf2),
        ('model', RandomForestClassifier())
    ])
}

# Optionally, print the pipelines to check
for model_name, pipeline in pipelines.items():
    print(f"{model_name} pipeline:", pipeline)

KNeighborsClassifier pipeline: Pipeline(steps=[('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standarisation',
                                                  StandardScaler(with_mean=False),
                                                  [0, 6])])),
                ('model', KNeighborsClassifier())])
SVC pipeline: Pipeline(steps=[('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standarisation',
                                                  StandardScaler(with_mean=False),
                                                  [0, 6])])),
                ('model', SVC())])
DecisionTreeClassifier pipeline: Pipeline(steps=[('encode',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_label',
                                                  OneHotEncoder(handle_unknown='ignor

In [38]:
for name, model in pipelines.items():
    model.fit(x_train.values, y_train)
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    print(f"{name} mode with accuracy: {score}")

C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


KNeighborsClassifier mode with accuracy: 0.9272727272727272


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


SVC mode with accuracy: 0.8681818181818182
DecisionTreeClassifier mode with accuracy: 0.9727272727272728


C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ADMIN\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


LogisticRegression mode with accuracy: 0.9


TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.